In [1]:
import os
import numpy as np
import pandas as pd
import zipfile
import timeit


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential

In [2]:
full_data = pd.read_csv("train.csv",encoding='utf-8')
test_data_X= pd.read_csv("test.csv",encoding='utf-8')
test_data_Y = pd.read_csv("test_labels.csv",encoding='utf-8')

In [3]:
full_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
print(f"There are {len(full_data)} observations in full data.")

There are 159571 observations in full data.


In [5]:
test_data_X.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
test_data_Y.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [7]:
test_dataframe = pd.merge(test_data_X, test_data_Y, how="inner", on="id")

# Remove all the rows having missing values (-1)
test_dataframe = test_dataframe[test_dataframe["toxic"] != -1].reset_index(drop=True)

test_dataframe.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
33876,876562ec312dc519,Jey: no borralo porque ni me acuerdo de la cla...,0,0,0,0,0,0
48950,c2eda7d9f7d9787b,Sourcing \n Please do not add content without ...,0,0,0,0,0,0
28363,7117be0ea929173e,It isn't you silly man,0,0,0,0,0,0
6798,1b73a5eaa446ca9f,Suspected sock puppetry: \n http://en.wikipedi...,0,0,0,0,0,0
20696,523ffa4c112d5623,"Alo idiote, jel umeš da čitaš? Samo ligaški na...",0,0,0,0,0,0


In [8]:
print(f"We have {len(test_dataframe)} observations in test data.")

We have 63978 observations in test data.


In [9]:
del test_data_X
del test_data_Y

In [10]:
full_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [11]:
full_data["comment_text"].values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [12]:
full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [13]:
train_data = tf.data.Dataset.from_tensor_slices(
    (
        full_data["comment_text"].tolist(),
        full_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

test_data = tf.data.Dataset.from_tensor_slices(
    (
        test_dataframe["comment_text"].tolist(),
        test_dataframe[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()
    )
)

In [14]:
batch_size = 30

train_data = train_data.cache().shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_data = test_data.cache().batch(batch_size)

In [15]:
max_tokens = 100000

start = timeit.default_timer()

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    standardize='lower_and_strip_punctuation',
    output_mode="int",
    output_sequence_length=1800
)

text_vectorization.adapt(full_data["comment_text"].tolist())

end = timeit.default_timer()

print(f"It took {end - start} seconds to adapt.")

It took 26.787046704999966 seconds to adapt.


In [16]:
del full_data
del test_dataframe

In [17]:
model = Sequential([
    text_vectorization,
    layers.Embedding(max_tokens+1, 32),
    layers.Bidirectional(layers.LSTM(32, return_sequences=False)),
    layers.Dense(256, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(6, activation="sigmoid")
])

In [18]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=metrics.BinaryAccuracy()
)

In [19]:
train_split = 0.8

train_data = train_data.take(int(len(train_data) * train_split))
valid_data = train_data.skip(int(len(train_data) * train_split))

In [20]:
print("The cardinality of train data is ", train_data.cardinality().numpy())
print("The cardinality of validation data is ", valid_data.cardinality().numpy())
print("The cardinality of test data is ", test_data.cardinality().numpy())

The cardinality of train data is  4256
The cardinality of validation data is  852
The cardinality of test data is  2133


In [21]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True,
        start_from_epoch=1,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=1e-5
    )
]

In [22]:
start = timeit.default_timer()

model_history = model.fit(
    train_data,
    epochs=10,
    verbose=1,
    callbacks=callbacks,
    validation_data=valid_data
).history

end = timeit.default_timer()

print(f"It took {end - start} seconds to train the model.")

Epoch 1/10
4256/4256 [==============================] - 537s 125ms/step - loss: 0.0629 - binary_accuracy: 0.9788 - val_loss: 0.0438 - val_binary_accuracy: 0.9833 - lr: 0.0010
Epoch 2/10
4256/4256 [==============================] - 429s 101ms/step - loss: 0.0446 - binary_accuracy: 0.9832 - val_loss: 0.0371 - val_binary_accuracy: 0.9858 - lr: 0.0010
Epoch 3/10
4256/4256 [==============================] - 425s 100ms/step - loss: 0.0389 - binary_accuracy: 0.9848 - val_loss: 0.0330 - val_binary_accuracy: 0.9882 - lr: 0.0010
Epoch 4/10
4256/4256 [==============================] - 426s 100ms/step - loss: 0.0334 - binary_accuracy: 0.9868 - val_loss: 0.0279 - val_binary_accuracy: 0.9895 - lr: 0.0010
Epoch 5/10
4256/4256 [==============================] - 421s 99ms/step - loss: 0.0291 - binary_accuracy: 0.9884 - val_loss: 0.0257 - val_binary_accuracy: 0.9911 - lr: 0.0010
Epoch 6/10
4256/4256 [==============================] - 418s 98ms/step - loss: 0.0256 - binary_accuracy: 0.9900 - val_loss: 0.

In [23]:
model.evaluate(test_data)

2133/2133 [==============================] - 91s 43ms/step - loss: 0.1152 - binary_accuracy: 0.9533


[0.11518029868602753, 0.9533433318138123]

In [24]:
text = "i will kill you"
predictions = model.predict(np.array([text])) > 0.5
prediction_text = ""
for i, col in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    prediction_text += f"{col:<{20}}: {predictions[0][i]:<{5}}\n"
print(prediction_text)

1/1 [==============================] - 1s 746ms/step
toxic               : 1    
severe_toxic        : 0    
obscene             : 0    
threat              : 1    
insult              : 0    
identity_hate       : 0    

